In [679]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: done

# All requested packages already installed.

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: done

# All requested packages already installed.

Libraries imported.


In [680]:
pip install beautifulsoup4 #library for pulling data out of HTML and XML files

Note: you may need to restart the kernel to use updated packages.


In [681]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [682]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [683]:
#downloading the page
import requests
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [684]:
#parsing the document
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content)

In [685]:
#Uncomment this line to check the doc but it is very long !
#print(soup.prettify())

In [686]:
#getting the columns index
col =[soup.select('table th')[0].get_text(),soup.select('table th')[1].get_text(),soup.select('table th')[2].get_text().strip()]
col

['Postcode', 'Borough', 'Neighbourhood']

In [687]:
#getting the content of the futur dataframe
l=len(soup.select('table tr td'))
i=0
postcode_list=[]
borough_list=[]
neigh_list=[]

while i<l :
    
    if soup.select('table tr td')[i].get_text().startswith('M'):
        postcode_list=postcode_list+[soup.select('table tr td')[i].get_text()]
        borough_list=borough_list+[soup.select('table tr td')[i+1].get_text()]
        neigh_list=neigh_list+[soup.select('table tr td')[i+2].get_text().strip()]
        i=i+3
    else : i=l 


In [688]:
#creating the dataframe from the lists extracted
ngh = pd.DataFrame([postcode_list, borough_list, neigh_list],index = col).T

In [689]:
#combining the neighborhoods
for i in range(ngh.shape[0]-1):
    codetest1=ngh['Postcode'][i]
    codetest2=ngh['Postcode'][i+1]
    if codetest1==codetest2 :
        ngh['Neighbourhood'][i+1]=ngh['Neighbourhood'][i] + ', ' + ngh['Neighbourhood'][i+1]

In [690]:
#droping the lines with no assigned borough and assigning neighborhood when not assigned
for i in range(0,ngh.shape[0]):
    test=ngh['Borough'][i]
    test2=ngh['Neighbourhood'][i]
    if test=='Not assigned':
        ngh=ngh.drop(i,0)
    elif test2=='Not assigned':
        ngh['Neighbourhood'][i]=ngh['Borough'][i]
        
ngh=ngh.reset_index(drop=True)

In [691]:
#droping multiple postcodes 
for i in range(0,ngh.shape[0]-1):
    test1=ngh['Postcode'][i]
    test2=ngh['Postcode'][i+1]
    if test1==test2:
        ngh=ngh.drop(i,0)
        

In [692]:
ngh=ngh.reset_index(drop=True)

In [693]:
#Getting the long and lat information
geodata=pd.read_csv('http://cocl.us/Geospatial_data')

In [694]:
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [695]:
#merging the dataframes
ngh=ngh.join(geodata.set_index('Postal Code'), on='Postcode')

In [696]:
#Here is the final dataframe !
ngh

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [697]:
#restricting the datas to the boroughs with Toronto in their names
toronto_data = ngh
for i in range(0,toronto_data.shape[0]):
    test=toronto_data['Borough'][i]
    if not ('Toronto' in test) :
        toronto_data=toronto_data.drop(i,0)

In [698]:
toronto_data=toronto_data.reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [699]:
#get the latitude and longitude values of Toronto

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [701]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

In [702]:
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [703]:
CLIENT_ID = 'ASLF5P1K0M20EXA3MTJRWVU5RO1SAGHGENVNY1OQM02MZ0MQ'
CLIENT_SECRET = 'UMRPESBCME1ABKBJDW5PKGCTUIDKPWWZEDWEOXWRVBHZZQ5R'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ASLF5P1K0M20EXA3MTJRWVU5RO1SAGHGENVNY1OQM02MZ0MQ
CLIENT_SECRET:UMRPESBCME1ABKBJDW5PKGCTUIDKPWWZEDWEOXWRVBHZZQ5R


In [704]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront, Regent Park are 43.6542599, -79.3606359.


In [705]:
LIMIT = 10 # limit of number of venues returned by Foursquare API
radius = 100 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ASLF5P1K0M20EXA3MTJRWVU5RO1SAGHGENVNY1OQM02MZ0MQ&client_secret=UMRPESBCME1ABKBJDW5PKGCTUIDKPWWZEDWEOXWRVBHZZQ5R&v=20180605&ll=43.6542599,-79.3606359&radius=100&limit=10'

In [706]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1f7d11e706550025e1ddeb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6551599009, 'lng': -79.35939430002972},
   'sw': {'lat': 43.653359899099996, 'lng': -79.36187749997029}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50760559e4b0e8c7babe2497',
       'name': 'Body Blitz Spa East',
       'location': {'address': '497 King Street East',
        'crossStreet': 'btwn Sackville St and Sumach St',
        'lat': 43.65473505045365,
        'lng': -79.35987433132891,
        'labe

In [707]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [708]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Body Blitz Spa East,Spa,43.654735,-79.359874
1,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
2,Massimo's Kitchen Studio,Italian Restaurant,43.654770,-79.359698
3,Sackville Playground,Park,43.654656,-79.359871


In [709]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [710]:
#create a function to repeat the same process to all the Boroughs

def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [711]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [712]:
print(toronto_venues.shape)
toronto_venues.head()

(93, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
1,"Harbourfront, Regent Park",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
2,"Harbourfront, Regent Park",43.654260,-79.360636,Massimo's Kitchen Studio,43.654770,-79.359698,Italian Restaurant
3,"Harbourfront, Regent Park",43.654260,-79.360636,Sackville Playground,43.654656,-79.359871,Park
4,"Ryerson, Garden District",43.657162,-79.378937,Balzac's Coffee,43.657854,-79.379200,Café


In [713]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",10,10,10,10,10,10
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",1,1,1,1,1,1
"Cabbagetown, St. James Town",1,1,1,1,1,1
Central Bay Street,4,4,4,4,4,4
"Chinatown, Grange Park, Kensington Market",5,5,5,5,5,5
Christie,1,1,1,1,1,1
"Commerce Court, Victoria Hotel",10,10,10,10,10,10
Davisville,8,8,8,8,8,8
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",3,3,3,3,3,3


In [714]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 52 uniques categories.


In [715]:
#analysing

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Brewery,Burger Joint,Café,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Deli / Bodega,Dessert Shop,Farmers Market,Fast Food Restaurant,Food Court,Gastropub,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,Hostel,Hotel,Italian Restaurant,Japanese Restaurant,Liquor Store,Mac & Cheese Joint,Nightclub,Park,Performing Arts Venue,Pizza Place,Pub,Record Shop,Restaurant,Roof Deck,Salad Place,Sandwich Place,Seafood Restaurant,Spa,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront, Regent Park",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Ryerson, Garden District",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [716]:
toronto_onehot.shape

(93, 53)

In [717]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Brewery,Burger Joint,Café,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Deli / Bodega,Dessert Shop,Farmers Market,Fast Food Restaurant,Food Court,Gastropub,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,Hostel,Hotel,Italian Restaurant,Japanese Restaurant,Liquor Store,Mac & Cheese Joint,Nightclub,Park,Performing Arts Venue,Pizza Place,Pub,Record Shop,Restaurant,Roof Deck,Salad Place,Sandwich Place,Seafood Restaurant,Spa,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.0,0.0,0.100000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000000,0.0,0.100000,0.000000,0.00,0.0,0.0,0.1,0.0,0.0,0.1,0.000000,0.0,0.000000,0.100000,0.000000,0.100000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.00,0.100000,0.000000,0.100,0.1,0.0,0.0,0.000,0.0,0.1,0.000000
1,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.00,0.000000,0.000000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
2,"Cabbagetown, St. James Town",0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.00,0.000000,0.000000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
3,Central Bay Street,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.750000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.25,0.000,0.00,0.000000,0.000000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
4,"Chinatown, Grange Park, Kensington Market",0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.200,0.2,0.000000,0.0,0.000000,0.000000,0.00,0.2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.00,0.000000,0.000000,0.000,0.0,0.0,0.2,0.000,0.0,0.0,0.000000
5,Christie,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.00,0.000000,0.000000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
6,"Commerce Court, Victoria Hotel",0.1,0.000000,0.1,0.0,0.000000,0.1,0.000000,0.000000,0.0,0.100,0.0,0.100000,0.0,0.000000,0.000000,0.00,0.0,0.1,0.0,0.0,0.0,0.0,0.100000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.0,0.10,0.100,0.00,0.000000,0.000000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
7,Davisville,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.125,0.0,0.125000,0.0,0.000000,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.125,0.00,0.000000,0.000000,0.125,0.0,0.0,0.0,0.125,0.0,0.0,0.000000
8,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.00,0.000,0.00,0.000000,0.333333,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
9,"Desig

In [718]:
toronto_grouped.shape

(21, 53)

In [719]:
#Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                           venue  freq
0                       Tea Room   0.1
1  Vegetarian / Vegan Restaurant   0.1
2            Japanese Restaurant   0.1
3                            Bar   0.1
4                          Hotel   0.1


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                   venue  freq
0  Performing Arts Venue   1.0
1            Art Gallery   0.0
2       Asian Restaurant   0.0
3           Liquor Store   0.0
4     Mac & Cheese Joint   0.0


----Cabbagetown, St. James Town----
                venue  freq
0  Italian Restaurant   1.0
1    Asian Restaurant   0.0
2        Liquor Store   0.0
3  Mac & Cheese Joint   0.0
4           Nightclub   0.0


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.75
1      Sandwich Place  0.25
2         Art Gallery  0.00
3  Mac & Cheese Joint  0.00
4           Nightclub  0.00


----Chinatown, Grange Par

In [720]:
#put into a Panda

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [721]:
#create the new dataframe 

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Hotel,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Food Court,Concert Hall,Steakhouse,Greek Restaurant,Bar,Tea Room
1,"CN Tower, Bathurst Quay, Island airport, Harbo...",Performing Arts Venue,Yoga Studio,Hostel,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market
2,"Cabbagetown, St. James Town",Italian Restaurant,Hostel,Gym / Fitness Center,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market
3,Central Bay Street,Coffee Shop,Sandwich Place,College Gym,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market
4,"Chinatown, Grange Park, Kensington Market",Thrift / Vintage Store,Liquor Store,Farmers Market,Café,Cocktail Bar,Yoga Studio,Concert Hall,Gym,Greek Restaurant,Gluten-free Restaurant


In [722]:
#cluster ngh

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

In [723]:
#new df with cluster and top 10 venues

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0.0,Italian Restaurant,Breakfast Spot,Park,Spa,Yoga Studio,Concert Hall,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.0,Art Gallery,Café,Concert Hall,Gym / Fitness Center,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,Japanese Restaurant,Hostel,Italian Restaurant,Performing Arts Venue,Coffee Shop,Deli / Bodega,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,Trail,Yoga Studio,College Gym,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [724]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        
        fill=True,
        
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [726]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0.0,Italian Restaurant,Breakfast Spot,Park,Spa,Yoga Studio,Concert Hall,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court
1,Downtown Toronto,0.0,Art Gallery,Café,Concert Hall,Gym / Fitness Center,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant
2,Downtown Toronto,0.0,Japanese Restaurant,Hostel,Italian Restaurant,Performing Arts Venue,Coffee Shop,Deli / Bodega,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub
3,East Toronto,0.0,Trail,Yoga Studio,College Gym,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market
5,Downtown Toronto,0.0,Coffee Shop,Sandwich Place,College Gym,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market
7,Downtown Toronto,0.0,Hotel,Sushi Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Food Court,Concert Hall,Steakhouse,Greek Restaurant,Bar,Tea Room
9,Downtown Toronto,0.0,Gym,Yoga Studio,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market
10,West Toronto,0.0,Yoga Studio,Record Shop,Asian Restaurant,Bar,Japanese Restaurant,Brewery,Mac & Cheese Joint,Coffee Shop,Pizza Place,Greek Restaurant
12,Downtown Toronto,0.0,Coffee Shop,Restaurant,Gym,Deli / Bodega,Yoga Studio,Concert Hall,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court
15,Downtown Toronto,0.0,Art Gallery,Café,Fast Food Restaurant,Pub,Coffee Shop,Sandwich Place,Seafood Restaurant,Gym,Bakery,Bookstore


In [727]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,1.0,Nightclub,Yoga Studio,Hostel,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market


In [728]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Downtown Toronto,2.0,Italian Restaurant,Hostel,Gym / Fitness Center,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market


In [729]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,3.0,Park,Yoga Studio,Hostel,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market
32,Downtown Toronto,3.0,Park,Yoga Studio,Hostel,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market


In [730]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Downtown Toronto,4.0,College Gym,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Greek Restaurant,Gluten-free Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market
